In [ ]:
!pip install cloudscraper

## VINPEARL_ATTRACTION

In [2]:
import cloudscraper
from bs4 import BeautifulSoup

url = "https://vinpearl.com/vi/23-dia-diem-du-lich-sai-gon-nghe-la-muon-xach-balo-len-va-di"
scraper = cloudscraper.create_scraper()
rq = scraper.get(url)
doc = BeautifulSoup(rq.text)

doc = doc.find('div', {"class":"container detail news-full"}).find('div', {"class":"content"})

In [3]:
img_tag = doc.find_all('img', {"alt":"dia diem du lich sai gon"})
leftover = doc.find('img', {"alt":"dia diem checkin sai gon"}).get("src")

img_src = []

for i in range(26):
  if (i == 0) or (i == 6) or (i == 8) or (i == 17):
    continue
  elif (i == 5) or (i == 7) or (i == 16):
    pair = [img_tag[i].get("src"), img_tag[i + 1].get("src")]
    img_src.append(pair)
    continue
  elif (i == 23):
    img_src.append(img_tag[i].get("src"))
    img_src.append(leftover)
  else:
    img_src.append(img_tag[i].get("src"))

In [4]:
p_tag = []
for item in doc.find_all('p')[1:-3]:
  if (">>>" not in item.get_text() and
      "Đặc biệt Vinpearl" not in item.get_text()):
    p_tag.append(item)

index_with_5_des = [4]
index_with_4_des = [11]
index_with_2_des = [2, 6, 7, 8, 10, 18, 19]

description = []
for i in range(23):
  if i in index_with_5_des:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text())
  elif i in index_with_4_des:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text())
  elif i in index_with_2_des:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text())
  else:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text())

In [5]:
data = []
h2_tag = doc.find_all('h2')
ul_tag = doc.find_all('ul')[:-1]
ul_tag.pop(5) # We don't want the value of ul_tag[5]
ul_tag.pop(5) # We don't want the value of ul_tag[6]
ul_tag.pop(5) # We don't want the value of ul_tag[7]
ul_tag.pop(5) # We don't want the value of ul_tag[8]

for i in range(23):
    data_itr = dict()

    piece = ul_tag[i].find_all('li')
    data_itr['Place'] = h2_tag[i].get_text().split(". ")[1]
    data_itr['Address'] = piece[0].get_text().split(": ")[1]
    data_itr['Time'] = piece[1].get_text().split(": ")[1]

    try:
      data_itr['Entrance Price'] = piece[2].get_text().split(": ")[1]
    except:
      data_itr['Entrance Price'] = "Miễn phí"
    
    data_itr['Image Source'] = img_src[i]
    data_itr['Description'] = description[i]
    
    data.append(data_itr)

In [ ]:
import pandas as pd
df_attraction = pd.DataFrame(data)
df_attraction.to_csv("CrawlData_vinpearl_attraction.csv", encoding="utf-8-sig", index = False)
#df_attraction

## DIVUI

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "http://divui.com/blog/72-dia-diem-du-lich-sai-gon-moi-ve-dem-mien-phi-tong-hop-tu-a-z/"
rq = requests.get(url)
doc = BeautifulSoup(rq.text)

doc = doc.find('div', {"class":"td-post-content"})

In [ ]:
p_tag = doc.find_all('p')[3:-5]
description = []

for item in p_tag:
  if (item.find('a') == None or "Cần Giờ là" in item.get_text()) and item.get("class") == None:
    description.append(item.get_text())

In [ ]:
figure_tag = doc.find_all('figure')
img_src  = []

for item in figure_tag:
  img_src.append(item.find('img').get("data-src"))

In [ ]:
h3_tag = doc.find_all('h3')
place = []

for item in h3_tag:
  place.append(item.get_text())

In [ ]:
h2_tag = doc.find_all('h2')[1:]
type_of_place = []

for item in h2_tag:
  type_of_place.append(item.get_text())

In [ ]:
ul_tag = doc.find_all('ul')[:-2]

li_fee = []
li_address = []

for item in ul_tag:
  if (item.find('li', {"style":"list-style-type: none;"})):
    continue
  li_tag = item.find_all('li')
  if (len(li_tag) == 2):
    li_fee.append(li_tag[0].get_text().split("vé:")[1][1:])
    li_address.append(li_tag[1].get_text().split(":")[1][1:])
  elif (len(li_tag) == 1 and "Giá vé" in li_tag[0].get_text()):
    li_fee.append(li_tag[0].get_text().split(": ")[1])
  else:
    li_address.append(li_tag[0].get_text().split(":")[1][1:])

index_without_address = [31, 37, 41, 42]
address = []
index = -1
for i in range(72):
  if i in index_without_address:
    address.append("No Information From Website")
  else:
    index = index + 1
    address.append(li_address[index])

index_with_fee = [0, 1, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                  18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 34, 35, 36]
ticket_price = []
index = -1
for i in range(72):
  if i in index_with_fee:
    index = index + 1
    ticket_price.append(li_fee[index])
  else:
    ticket_price.append("No Ticket Needed")

In [ ]:
data = []
for i in range(72):
  data_itr = dict()
  data_itr['Place'] = place[i]
  data_itr['Address'] = address[i]

  if (i <= 14):
    data_itr['Type'] = type_of_place[0]
  elif (i <= 31):
    data_itr['Type'] = type_of_place[1]
  elif (i <= 42):
    data_itr['Type'] = type_of_place[2]
  else:
    data_itr['Type'] = type_of_place[3]
    
  data_itr['Ticket Price'] = ticket_price[i]
  data_itr['Image Source'] = img_src[i]
  data_itr['Description'] = description[i]
  
  data.append(data_itr)

In [ ]:
import pandas as pd
df_divui = pd.DataFrame(data)
df_divui.to_csv("CrawlData_divui.csv", encoding="utf-8-sig", index = False)
#df_divui

## VINPEARL_CAFE

In [ ]:
import cloudscraper
from bs4 import BeautifulSoup

url = "https://vinpearl.com/vi/top-20-quan-cafe-dep-o-sai-gon-nen-ghe-du-chi-mot-lan"
scraper = cloudscraper.create_scraper()
rq = scraper.get(url)
doc = BeautifulSoup(rq.text)
doc = doc.find(class_="content")

In [ ]:
place = []
for item in doc.find_all('h3'):
  place.append(item.get_text())

In [ ]:
address = []
time = []
ul_tag = doc.find_all('ul')[:-2]
index_without_address_time = [12, 13]

i = -1
for item in ul_tag:
  i = i + 1
  if (i in index_without_address_time):
    address.append("No Information From Website")
    time.append("No Information From Website")
    address.append("No Information From Website")
    time.append("No Information From Website")
    li_tag = item.find_all('li')
    address.append(li_tag[0].get_text().split(": ")[1])
    time.append(li_tag[1].get_text().split("Giờ mở cửa:")[1][1:])
    i = 100
  else:
    li_tag = item.find_all('li')
    address.append(li_tag[0].get_text().split(": ")[1])
    time.append(li_tag[1].get_text().split("Giờ mở cửa:")[1][1:])

In [ ]:
img_src = []
index_with_3_cap = [12, 13]
index_with_2_cap = [19]
index_with_both_type = [15, 16]
index_with_3_uncap = [4]
index_with_2_uncap = [17]

cap = [item for item in doc.find_all('img', {"alt":"quán cafe đẹp ở Sài Gòn"})]
uncap = [item for item in doc.find_all('img', {"alt":"quán cafe đẹp ở sài gòn"})]

for i in range(20):
  if i in index_with_3_cap:
    group = [cap.pop(0), cap.pop(0), cap.pop(0)]
  elif i in index_with_2_cap:
    group = [cap.pop(0), cap.pop(0)]
  elif i in index_with_both_type:
    group = [cap.pop(0), uncap.pop(0)]
  elif i in index_with_3_uncap:
    group = [uncap.pop(0), uncap.pop(0), uncap.pop(0)]
  elif i in index_with_2_uncap:
    group = [uncap.pop(0), uncap.pop(0)]
  else:
    group = [cap.pop(0), uncap.pop(0), uncap.pop(0)]
  img_src.append(group)

img_src_copy = img_src.copy()
img_src.clear()
for item in img_src_copy:
  temp = []
  for each in item:
    temp.append(each.get("src"))
  img_src.append(temp)

In [ ]:
p_tag = []
for item in doc.find_all('p')[1:-10]:
  if ("42 Thảo Điền" not in item.get_text() and
      item.find('a') == None or
      "chợ Bến Thành" in item.get_text() or
      "phố đi bộ Nguyễn Huệ" in item.get_text() or
      "Dinh Độc Lập" in item.get_text() or
      "phố Tây Bùi Viện" in item.get_text()):
    p_tag.append(item)

description = []
index_with_3_des = [4, 12, 13]
index_with_1_des = [15, 19]
for i in range(20):
  if i in index_with_3_des:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text())
  elif i in index_with_1_des:
    description.append(p_tag.pop(0).get_text())
  else:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text())

In [ ]:
data = []
for i in range(20):
  data_itr = dict()
  data_itr['Place'] = place[i]
  data_itr['Address'] = address[i]
  data_itr['Time'] = time[i]
  data_itr['Image Source'] = img_src[i]
  data_itr['Description'] = description[i]

  data.append(data_itr)

In [ ]:
import pandas as pd
df_cafe = pd.DataFrame(data)
df_cafe.to_csv("CrawlData_vinpearl_cafe.csv", encoding="utf-8-sig", index = False)
#df_cafe

## TOPLIST

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://toplist.vn/top-list/quan-an-uong-tren-cao-view-dep-nhat-sai-gon-12068.htm"
url_home = "https://toplist.vn"
rq = requests.get(url)
doc = BeautifulSoup(rq.text)

In [ ]:
infos = doc.find_all(class_="media-body")[:10]
# Tạo các danh sách để lưu thông tin
place = []
address = []
hotline = []
fanpage = []
website = []
price = []
time = []
description = []
img_src = []

i = -1
for item in infos:
  # Lưu tên và link ảnh minh họa
  place.append(item.find('h3').get_text())
  figure_tag = item.find_all('figure')
  pair = []
  for each in figure_tag:
    pair.append(url_home + each.find("img").get("src"))
  img_src.append(pair)
  # Lưu các thông tin còn lại
  p_tag = item.find_all('p')
  description.append(p_tag[0].get_text() + p_tag[2].get_text())
  i = i + 1
  j = -1
  for each in p_tag:
    j = j + 1
    if (i == 2 and j == len(p_tag) - 1):
        time.append("No Information From Website")
        price.append("No Information From Website")
    if (i != 6):
      if ("Địa chỉ:" in each.get_text()):
        address.append(each.get_text().split(": ")[1])
      elif ("Điện thoại:" in each.get_text()):
        hotline.append(each.get_text().split(": ")[1])
      elif ("Fanpage:" in each.get_text()):
        fanpage.append(each.get_text().split(": ")[1])
      elif ("Website:" in each.get_text()):
        website.append(each.get_text().split(": ")[1])
      elif ("Giá tham khảo:" in each.get_text()):
        price.append(each.get_text().split(": ")[1])
        time.append("No Information From Website")
      elif ("Giờ mở cửa:" in each.get_text()):
        time.append(each.get_text().split(": ")[1])
        price.append("No Information From Website")
    else:
      if ("Địa chỉ:" in each.get_text()):
        address.append(each.get_text().split(": ")[1].split("Điện thoại")[0])
        hotline.append(each.get_text().split(": ")[2].split("Fanpage")[0])
        fanpage.append(each.get_text().split(": ")[3].split("Website")[0])
        website.append(each.get_text().split(": ")[4].split("Giờ mở cửa")[0])
        time.append(each.get_text().split(": ")[-1])
        price.append("No Information From Website")    


In [ ]:
data = []
for i in range(10):
  data_itr = dict()
  data_itr['Place'] = place[i]
  data_itr['Address'] = address[i]
  data_itr['Hotline'] = hotline[i]
  data_itr['Fanpage'] = fanpage[i]
  data_itr['Website'] = website[i]
  data_itr['Time'] = time[i]
  data_itr['Price'] = price[i]
  data_itr['Image Source'] = img_src[i]
  data_itr['Description'] = description[i]

  data.append(data_itr)

In [ ]:
import pandas as pd
df_toplist = pd.DataFrame(data)
df_toplist.to_csv("CrawlData_toplist.csv", encoding="utf-8-sig", index = False)
#df_toplist

## DULICHTODAY

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://dulichtoday.vn/kham-pha-sai-gon/dia-diem-sai-gon/dia-diem-to-chuc-su-kien-sai-gon.html'
rq = requests.get(url)
doc = BeautifulSoup(rq.text)

doc = doc.find(class_="container-wrapper post-content tie-standard")

In [ ]:
place = []
for item in doc.find_all('h4'):
  string = item.get_text().replace('\xa0', ' ')
  place.append(string.split('. ')[1])

In [ ]:
img_src = []
for item in doc.find_all('img')[2:]:
  if (item.get('data-src') not in img_src and
      item.get('data-src') != None):
    img_src.append(item.get('data-src'))

img_src_copy = img_src.copy()
img_src.clear()
while img_src_copy:
  group = [img_src_copy.pop(0), img_src_copy.pop(0)]
  img_src.append(group)

In [ ]:
p_tag = doc.find_all('p')[5:-1]
p_tag.pop(-20)
len(p_tag)

for i, item in enumerate(p_tag):
  if ("Sau đây là" in item.get_text() or
      "Dưới đây là" in item.get_text()):
    p_tag.pop(i)

description = []
for i in range(14):
  description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text())

In [ ]:
ul_tag = doc.find_all('ul')[18:]
contact_tag = ul_tag[::3]
pros_tag = ul_tag[1::3]
cons_tag = ul_tag[2::3]

In [ ]:
address = []
price = []
size = []
website = []

for item in contact_tag:
  li_tag = item.find_all('li')
  address.append(li_tag[0].get_text().split(': ')[1])
  price.append(li_tag[2].get_text().split(': ')[1])
  size.append(li_tag[3].get_text().split(': ')[1])
  website.append(li_tag[4].find('a').get('href'))

In [ ]:
pros = []
for item in pros_tag:
  string = []
  li_tag = item.find_all('li')
  for each in li_tag:
    string.append(each.get_text())
  pros.append(string)

cons = []
for item in cons_tag:
  string = []
  li_tag = item.find_all('li')
  for each in li_tag:
    string.append(each.get_text())
  cons.append(string)

In [ ]:
data = []
for i in range(14):
  data_itr = dict()
  data_itr['Place'] = place[i]
  data_itr['Address'] = address[i]
  data_itr['Price'] = price[i]
  data_itr['Size'] = size[i]
  data_itr['Website'] = website[i]
  data_itr['Pros'] = pros[i]
  data_itr['Cons'] = cons[i]
  data_itr['Image Source'] = img_src[i]
  data_itr['Description'] = description[i]

  data.append(data_itr)

In [ ]:
import pandas as pd
df_dulichtoday = pd.DataFrame(data)
df_dulichtoday.to_csv("CrawlData_dulichtoday.csv", encoding="utf-8-sig", index = False)
#df_dulichtoday